# Explore verejne zakazky
elastic search should be running for this

In [1]:
from elasticsearch import Elasticsearch
from exploretools import Explorer
es = Elasticsearch()
ex = Explorer(es)

In [2]:
index = 'verejnezakazky_2018-02-13'
type_vz = 'verejnazakazka'
type_pz = 'profilzadavatele'
docs_vz = {'index': index, 'doc_type': type_vz}
docs_pz = {'index': index, 'doc_type': type_pz}

## Verejne zakazky

In [3]:
print("Total count:")
vz_total = es.count(**docs_vz)['count']
vz_total

Total count:


185790

In [6]:
ds = es.search(**docs_vz)['hits']['hits']
ds[0]

{'_id': '8700CC9E1FF5D48FD7F29DC100B35B85',
 '_index': 'verejnezakazky_2018-02-13',
 '_score': 1.0,
 '_source': {'cPV': ['16000000-5'],
  'dataset': 'VVZ-2006',
  'datumUverejneni': '2014-09-09T00:00:00',
  'displayId': 0,
  'dodavatele': [],
  'dokumenty': [],
  'evidencniCisloZakazky': '496393',
  'formulare': [{'cislo': '7452010096393',
    'druh': 'F52',
    'onlyOnProfile': False,
    'uRL': '',
    'zverejnen': '2014-09-09T00:00:00'}],
  'id': '8700CC9E1FF5D48FD7F29DC100B35B85',
  'kriteria': [],
  'lastUpdated': '0001-01-01T00:00:00',
  'posledniZmena': '2014-09-09T00:00:00',
  'zadavatel': {'iCO': '62698044', 'jmeno': 'Pavel Francl'}},
 '_type': 'verejnazakazka'}

Is `evidencniCisloZakazky` and `id` unique? 

In [107]:
def print_n_unique(field):
    print(f"{field}: ")
    ex.n_unique_exact(docs_vz, field)
print_n_unique('evidencniCisloZakazky')
print_n_unique('id')
print_n_unique('displayId')

evidencniCisloZakazky: 
Went through 185790 entries.
Found 180171 unique entries.
id: 
Went through 185790 entries.
Found 185790 unique entries.
displayId: 
Went through 185790 entries.
Found 1 unique entries.


In [116]:
cnts = ex.value_counts_exact(docs_vz, 'evidencniCisloZakazky')
cnts.head()

Went through 185790 entries.
Found 180171 unique entries.


638866    2
638683    2
638650    2
404824    2
638656    2
dtype: int64

Check how do the duplicit entries look like.

In [120]:
def get_by_field(docs_param, field, value):
    body = {'query': {'term': {field: value}}}
    return es.search(**docs_param, body=body)['hits']['hits']
get_by_field(docs_vz, 'evidencniCisloZakazky', 638656)

[{'_id': 'F00975C4A9CC72DD0087E6622F416D86',
  '_index': 'verejnezakazky_2018-02-13',
  '_score': 10.115287,
  '_source': {'cPV': ['35125300-2'],
   'dataset': 'VVZ-2016',
   'datumUverejneni': '2016-10-27T00:00:00+02:00',
   'datumUzavreniSmlouvy': '2016-10-14T00:00:00+02:00',
   'displayId': 0,
   'dodavatele': [{'iCO': '25229397', 'jmeno': 'SUPTel a.s.'}],
   'dokumenty': [],
   'evidencniCisloZakazky': '638656',
   'formulare': [{'cislo': 'F2016-001786',
     'druh': 'F03',
     'onlyOnProfile': False,
     'uRL': '',
     'zverejnen': '2016-10-27T00:00:00'}],
   'id': 'F00975C4A9CC72DD0087E6622F416D86',
   'konecnaHodnotaBezDPH': 790942,
   'konecnaHodnotaMena': 'CZK',
   'kriteria': [],
   'lastUpdated': '2016-10-27T00:00:00',
   'nazevZakazky': 'Rozšíření Městského kamerového systému v\xa0Plzni – lokalita Severní předměstí\n',
   'odhadovanaHodnotaBezDPH': 811667,
   'odhadovanaHodnotaMena': 'CZK',
   'popisZakazky': 'Předmětem plnění veřejné zakázky v rámci tohoto zadávacího ří

OK, it looks like conversion from dataset VVZ-2006 to VVZ-2016.
Lets check the dataset counts.

In [109]:
cnts = ex.value_counts_exact(docs_vz, 'dataset')
cnts.head()

Went through 185790 entries.
Found 2 unique entries.


VVZ-2006    157104
VVZ-2016     28686
dtype: int64

## Profily zadavatele

In [14]:
print("Total count:")
es.count(**docs_pz)['count']

Total count:


18859

In [15]:
ds = es.search(**docs_pz)['hits']['hits']
ds[0]

{'_id': 'VVZ-2016-403861',
 '_index': 'verejnezakazky_2018-02-13',
 '_score': 1.0,
 '_source': {'dataSet': 'VVZ-2016',
  'datumUverejneni': '2014-12-09T00:00:00+01:00',
  'evidencniCisloFormulare': 'F2014-552076',
  'evidencniCisloProfilu': '403861',
  'id': 'VVZ-2016-403861',
  'lastAccess': '2018-02-08T19:31:56.7769608+01:00',
  'lastAccessResult': 0,
  'url': 'http://enjoy-trade.profilzadavatele.cz/',
  'zadavatel': {'iCO': '26950707', 'jmeno': 'Enjoy Trade, s.r.o.'}},
 '_type': 'profilzadavatele'}